In [2]:
import array

def pmn():
    pass

def my_r(n):
    i = 0
    while i < n:
        yield i
        i += 1

def get_objects():
    arr = [ pmn,
            str.center,
              bytearray(b'str'),
              {11,22,33,44},
              ValueError(),
              Exception(),
              len,
              iter([111,222,333]),
              frozenset({11,22,33,44}), memoryview(b'123'),
              object(),
              type(object()),
              123,
              b'123',
              (9,10,21),
              range(123),
              array,
              iter({'123':'123'}),
              {'123':'123'},
              '123',
              123.0,
              None,
              123+123j,
              False,
              [],
              my_r(2) ]
    
    return arr


In [3]:
get_objects()


[<function __main__.pmn()>,
 <method 'center' of 'str' objects>,
 bytearray(b'str'),
 {11, 22, 33, 44},
 ValueError(),
 Exception(),
 <function len(obj, /)>,
 frozenset({11, 22, 33, 44}),
 object,
 123,
 b'123',
 (9, 10, 21),
 range(0, 123),
 <module 'array' (built-in)>,
 {'123': '123'},
 '123',
 123.0,
 None,
 (123+123j),
 False,
 [],
 <generator object my_r at 0x0000014E80D33A50>]

In [4]:
class list_divider(list):
    def __truediv__(self, number):
        if number <= 0:
            raise ValueError()
        if type(number) is not int:
            raise TypeError()
        first, second = divmod(len(self), number)
        result = [self[idx * first  +  min(idx, second) : (idx + 1) * first  +  min(idx + 1, second)] for idx in range(number) ]
        return result

In [75]:
class graph_partition:
    def __init__(self, proc_id, n_proc):
        self.proc_id = proc_id
        self.n_proc = n_proc
        self.length = 0
        self.idx = 0
        self.jdx = 0
    
    def fit(self, n):
        if n <= 0:
            return
        self.n = n
        all_pair = int(n * (n - 1) / 2)
        self.length = all_pair // self.n_proc
        start = self.proc_id * self.length
        
        if self.proc_id >= all_pair % self.n_proc:
            start += all_pair % self.n_proc
        else:
            self.length += 1
            start += self.proc_id

        for kdx in range(n):
            if start >= 0: start -= (n - kdx - 1)
            else: break

        self.idx = kdx - 1
        self.jdx = start + n

    def __len__(self): 
        return self.length 
    
    def __iter__(self):
        self.iter = 0
        return self

    def __next__(self):
        if self.iter >= self.length: raise StopIteration
        self.iter += 1
        if self.jdx < self.n:
            self.jdx += 1
            return (self.idx, self.jdx - 1)
        else:
            self.jdx = self.idx + 3
            self.idx += 1
            return (self.idx, self.jdx - 1)

In [76]:
graph = graph_partition(2,4) # 0,2 2,4
graph.fit(10) # 3 10

for edge in graph:
    print(edge)

(2, 9)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(4, 5)
(4, 6)
(4, 7)
(4, 8)


In [5]:
import numpy as np

class score_filter:
    def __init__(self, min_score, max_score):
        self.min_score = min_score
        self.max_score = max_score

    def fit(self, x, y, iterable):
        self.iterable = iterable
        self.x_cumsum = np.cumsum(x, dtype=np.float32)
        self.y_cumsum = np.cumsum(y, dtype=np.float32)
        self.sum = self.x_cumsum[-1] * self.y_cumsum[-1]
        self.result = []
        for idx, jdx in self.iterable:
            if self.min_score < (self.x_cumsum[idx] * self.y_cumsum[jdx] / self.sum) and (self.x_cumsum[idx] * self.y_cumsum[jdx] / self.sum) < self.max_score:
                self.result.append((idx, jdx))
    
    def __len__(self):
        return len(self.result)

    def __iter__(self):
        self.current = 0
        return self

    def __next__(self):
        if self.current + 1 > len(self.result):
            raise StopIteration
        self.current += 1
        return self.result[self.current-1] 


In [6]:
filter = score_filter(0, 1)

filter.fit([23,1,2,3], [33,1,2,3], [(0,0),(0,1),(0,1), (1,1)])

for elem in filter:
    print(elem)

(0, 0)
(0, 1)
(0, 1)
(1, 1)


In [260]:
days_in_m = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31 }

def parse_date(string: str):
    isValid = True

    if "//" in string:
        isValid = False
    string = string.strip()
    
    if len(string) >= 2:
        if string[0] == '/' or string[-1] == '/':
            isValid = False

    parsed_values = string.split("/")
    parsed_values = tuple(elem.replace(" ", "") for elem in parsed_values if elem.isdigit())
    if len(parsed_values) != 3:
        isValid = False
        return 0,0,0,isValid

    day = int(parsed_values[0])
    mounth = int(parsed_values[1])
    year = int(parsed_values[2])

    if not day > 0 or not day <= 31 or not mounth > 0 or not mounth <= 12 or not year >= 1982 or not year <= 2022:
        isValid = False
        return day, mounth, year, isValid
    
    if not day <= days_in_m[mounth]:
        isValid = False

    return day, mounth, year, isValid

def common_tc_dates(cyclone_dates_1 : list[str], 
                    cyclone_dates_2 : list[str]) -> dict:
    res ={
        "any_year": 0,
        "both_years": 0,
        "only_one_year": 0,
        "only_first_year": 0,
        "only_second_year": 0, 
        "none_of_years": 365 }

    days1 = []
    date_full1 = []
    days2 = []

    for elem in cyclone_dates_1:
        day, mounth, year, isValid = parse_date(elem)
        print(day, mounth, year, isValid)
        if not isValid:
            continue
        date = (day, mounth)
        date_full = (day, mounth, year)
        if not date in days1:
            days1.append(date)
            date_full1.append(date_full)
            res["any_year"] += 1
            res["only_one_year"] += 1
            res["only_first_year"] += 1
            res["none_of_years"] -= 1
    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    for elem in cyclone_dates_2:
        day, mounth, year, isValid = parse_date(elem)
        print(day, mounth, year, isValid)
        if not isValid:
            continue
        date = (day, mounth)
        date_full = (day, mounth, year)
        if not date in days2:
            days2.append(date)
            if not date in days1:
                res["any_year"] += 1
                res["only_one_year"] += 1
                res["only_second_year"] += 1
                res["none_of_years"] -= 1
            else:
                res["only_one_year"] -= 1
                res["only_first_year"] -= 1
                res["both_years"] += 1
    res["only_one_year"] = res["only_first_year"] + res["only_second_year"]
    res["none_of_years"] = 365 - res["any_year"]
    res["both_years"] = res["any_year"] - res["only_one_year"]
    return res

In [267]:
days_in_m = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31 }

def parse_date(string: str):
    isValid = True

    parsed_values = string.strip().replace(" ", "").split("/")

    if len(parsed_values) != 3:
        isValid = False
        return 0,0,0,isValid
    else:
        day = int(parsed_values[0])
        mounth = int(parsed_values[1])
        year = int(parsed_values[2])

    if not day >= 1 or not day <= 31 or not mounth >= 1 or not mounth <= 12 or not year >= 1982 or not year <= 2022:
        isValid = False
        return day, mounth, year, isValid
    
    if not day <= days_in_m[mounth]:
        isValid = False

    return day, mounth, year, isValid

def common_tc_dates(cyclone_dates_1 : list[str], 
                    cyclone_dates_2 : list[str]) -> dict:
    res = { "any_year": 0,
            "both_years": 0,
            "only_one_year": 0,
            "only_first_year": 0,
            "only_second_year": 0, 
            "none_of_years": 365 }

    days1 = []
    days2 = []

    for elem in cyclone_dates_1:
        day, mounth, year, isValid = parse_date(elem)
        if not isValid:
            continue
        date = (day, mounth)
        if not date in days1:
            days1.append(date)
            res["any_year"] += 1
            res["only_first_year"] += 1
    for elem in cyclone_dates_2:
        day, mounth, year, isValid = parse_date(elem)
        if not isValid:
            continue
        date = (day, mounth)
        if not date in days2:
            days2.append(date)
            if not date in days1:
                res["any_year"] += 1
                res["only_second_year"] += 1
            else:
                res["only_first_year"] -= 1

    res["only_one_year"] = res["only_first_year"] + res["only_second_year"]
    res["none_of_years"] = 365 - res["any_year"]
    res["both_years"] = res["any_year"] - res["only_one_year"]
    return res

In [268]:
# cyclone_dates_1 = ["04/01/2014", "05/01/2014"]
# cyclone_dates_2 = ["06/01/2015", "04/01/2016"]

cyclone_dates_1 = []
cyclone_dates_2 = [] # "34/01/2014"


for m in range(1, 13):
    for d in range(1, days_in_m[m]+1):
        cyclone_dates_1.append(str(d)+"/"+str(1)+"/"+str(1989))
for m in range(1, 13):
    for d in range(1, 31+1):
        for y in range(2012, 2020):
            cyclone_dates_2.append(str(d)+"/"+str(m)+"/"+str(y))

# cyclone_dates_2 = cyclone_dates_1.copy()
# cyclone_dates_1.remove("1/4/1999")
# print(cyclone_dates_1)
cyclone_dates_1 = ["31/01/2016", "31/12/2013"]
cyclone_dates_2 = ["30/11/2011", "31/01/2016", "22/02/1997", "12/05/1997"] 

common_tc_dates(cyclone_dates_1, cyclone_dates_2)

{'any_year': 5,
 'both_years': 1,
 'only_one_year': 4,
 'only_first_year': 1,
 'only_second_year': 3,
 'none_of_years': 360}

In [21]:
import numpy as np
  
# creating the array
arr = np.array([10, 32, 30, 50, 20, 82, 91, 45])
  
#  printing arr
print("arr = {}".format(arr))
  
#  looking for value 30 in arr and storing its index in i
i = np.where(arr == 100000)
print("i = {}".format(i))

arr = [10 32 30 50 20 82 91 45]
i = (array([], dtype=int64),)


In [1]:
import numpy as np
import math

def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    base = np.copy(movie)
    N, M, T = movie.shape
    Flags = np.ones((N,M))
    _B = np.zeros((T,))


    WS = mean_window_size
    Left, Right = math.floor(mean_window_size / 2), math.ceil(mean_window_size/2)
    w_sizes = np.asarray([min(T, t+Right) - max(0,t-Left) for t in range(T)])


    for _ in range(num_iterations):
        for i,j in np.ndindex((N,M)):
            if Flags[i][j]:
    
                mins = np.minimum(base[i][j], movie[i][j])
                
                base[i][j] = np.convolve(mins, np.ones(mean_window_size), 'same') / w_sizes
                
                mins = np.minimum(base[i][j], movie[i][j])

                tmp = np.mean((mins - base[i][j]) ** 2)


                D = np.sqrt(tmp * T / (T - 1))
                if D < f_noise_sigma[i][j]:
                    Flags[i][j] = 0
                    if np.sum(Flags) == 0:
                        return base
    return base
